Decorate the inertia_data file

In [10]:
import pandas as pd
import tools.tools as t
from tqdm import tqdm 

In [1]:
halo_list = pd.read_csv('lists/list_halo.dat',
                        delim_whitespace=True,
                        skiprows=1,
                        names=['id', 'level', 'mass', 'x', 'y', 'z', 'r']).set_index('id')

In [4]:
unit, ncol, nrow = t.io.read_list_header('lists/list_kingal_00782.dat')
gal_dynamics = pd.DataFrame(t.io.read_list_data_reals(unit, ncol, nrow),
                            columns=['id', 'vtheta', 'dvx', 'dvy', 'dvz', 'mass', 'x', 'y', 'z']).set_index('id')

gd = gal_dynamics
gal_dynamics['sigma_over_vtheta'] = 1./3*np.sqrt((gd.dvx**2 + gd.dvy**2 + gd.dvz**2)) / np.abs(gd.vtheta)
del gd

In [5]:
association = pd.read_csv('lists/associated_halogal_782.dat', delim_whitespace=True, skiprows=1,
                          names=['halo_id', 'level', 'halo_mass', 'gal_id', 'gal_mass']).set_index('halo_id')

In [6]:
inertia_data = pd.read_csv('data/halo_kin.00002.1e10_m.dat', delim_whitespace=True, dtype=np.float,
                           engine='c', index_col=0)

print('Found {} halo.'.format(inertia_data['mass'].size))

Found 280219 halo.


In [7]:
def get_sov():
    halo_with_galaxy = set(association.index)
    inertia_data_halos = set(inertia_data.index)
    common = halo_with_galaxy.intersection(inertia_data_halos)
    gal_ids = association.loc[common, 'gal_id'].as_matrix()
    sov = gal_dynamics.loc[gal_ids, 'sigma_over_vtheta'].as_matrix()
    return common, sov
ids, sov = get_sov()

inertia_data['host_gal'] = False
inertia_data.loc[ids, 'sigma_over_vtheta'] = sov
inertia_data.loc[ids, 'host_gal'] = True

In [8]:
%%cython
cimport numpy as np
import numpy as np
cimport cython

@cython.boundscheck(False)
cdef np.ndarray[double, ndim=2] cy_compute_tensor(np.ndarray[double, ndim=1] xx, np.ndarray[double, ndim=1] xy, 
                   np.ndarray[double, ndim=1] xz, np.ndarray[double, ndim=1] yy,
                   np.ndarray[double, ndim=1] yz, np.ndarray[double, ndim=1] zz, tqdm):
    cdef int i, j
    cdef np.ndarray[double, ndim=2] I
    cdef np.ndarray[double, ndim=2] eva = np.zeros((len(xx), 3))
    cdef np.ndarray[double, ndim=1] tmp

    for i in tqdm(range(len(xx))):
        I = np.zeros((3, 3))
        I[0,0] = xx[i]
        I[0,1] = xy[i]
        I[1,0] = xy[i]
        I[0,2] = xz[i]
        I[2,0] = xz[i]
        I[1,1] = yy[i]
        I[1,2] = yz[i]
        I[2,1] = yz[i]
        I[2,2] = zz[i]

        tmp = np.linalg.eigvalsh(I)
        for j in range(3):
            eva[i, j] = tmp[j]
    return eva

def compute_tensor(d, tqdm=None):
    cdef np.ndarray[double, ndim=1] xx, xy, xz, yy, yz, zz 
    xx, xy, xz, yy, yz, zz = np.array([d['xx'], d['xy'], d['xz'], d['yy'], d['yz'], d['zz']], dtype=np.float)
    if tqdm == None:
        tqdm = lambda i: i
    return cy_compute_tensor(xx, xy, xz, yy, yz, zz, tqdm)                

In [11]:
eva = compute_tensor(inertia_data, tqdm=tqdm)

100%|██████████| 280219/280219 [00:13<00:00, 20112.62it/s]


In [12]:
def get_sov():
    halo_with_galaxy = set(association.index)
    inertia_data_halos = set(inertia_data.index)
    common = halo_with_galaxy.intersection(inertia_data_halos)
    gal_ids = association.loc[common, 'gal_id'].as_matrix()
    sov = gal_dynamics.loc[gal_ids, 'sigma_over_vtheta'].as_matrix()
    return common, sov
ids, sov = get_sov()

inertia_data['host_gal'] = False
inertia_data.loc[ids, 'sigma_over_vtheta'] = sov
inertia_data.loc[ids, 'host_gal'] = True

In [13]:
inertia_data['l0'] = np.sqrt(eva[:, 0])
inertia_data['l1'] = np.sqrt(eva[:, 1])
inertia_data['l2'] = np.sqrt(eva[:, 2])

In [14]:
inertia_data[['log(mass)', 'log(l0)', 'log(l1)', 'log(l2)', 'log(sigma_over_vtheta)']] = (
    np.log10(inertia_data[['mass', 'l0', 'l1', 'l2', 'sigma_over_vtheta']])
    )

In [17]:
halos = t.read_brick('/data52/Horizon-AGN/TREE_DM_celldx2kpc_SC0.9r/tree_bricks782', low_mem=['nparts', 'members', 'idh', 'hlevel'])
l = list(tqdm(halos.it))
halos_props = pd.DataFrame([{'hlevel': h['hlevel'], 'idh':h['idh'][0], 'nparts': h['nparts'][0]} for h in l]).set_index('idh')
halo_members = [{'idh': h['idh'][0], 'members': h['members']} for h in l]
del l

321713it [00:49, 6530.94it/s]


In [18]:
to_set = set(inertia_data.index)
from_set = set(halos_props.index)

inter = list(to_set.intersection(from_set))
inertia_data.loc[inter, 'hlevel'] = halos_props.loc[inter, 'hlevel']

In [19]:
inertia_data['prolateness'] = inertia_data.eval('l2/sqrt(l1*l0)')
inertia_data['log(prolateness)'] = np.log10(inertia_data['prolateness'])

inertia_data['oblateness'] = inertia_data.eval('sqrt(l2*l1)/l0')
inertia_data['log(oblateness)'] = np.log10(inertia_data['oblateness'])

inertia_data['prod_li'] = inertia_data.eval('l2*l1*l0')
inertia_data['log(prod_li)'] = np.log10(inertia_data['prod_li'])

In [20]:
bins = [0, 0.5, 1.5, np.infty]
labels = ['elliptical', 'intermediate', 'spiral']
inertia_data['vtheta_over_sigma'] = 1./inertia_data['sigma_over_vtheta']
inertia_data['galaxy_type'] = pd.cut(inertia_data['vtheta_over_sigma'], bins, labels=labels)

In [22]:
inertia_data.columns

Index([u'mass', u'x', u'y', u'z', u'xx', u'xy', u'xz', u'yy', u'yz', u'zz',
       u'host_gal', u'sigma_over_vtheta', u'l0', u'l1', u'l2', u'log(mass)',
       u'log(l0)', u'log(l1)', u'log(l2)', u'log(sigma_over_vtheta)',
       u'hlevel', u'prolateness', u'log(prolateness)', u'oblateness',
       u'log(oblateness)', u'prod_li', u'log(prod_li)', u'vtheta_over_sigma',
       u'galaxy_type'],
      dtype='object')

Remove some columns that cannot be exported to hdf format

In [24]:
inertia_data_cleaned = inertia_data.drop('galaxy_type', 1)

In [28]:
inertia_data_cleaned.to_hdf('data/halo_kin.00002.1e10_m.hdf', 'inertia_data')
inertia_data.to_csv('data/halo_kin.00002.1e10_m_completed.csv', delim_whitespace=True)

In [30]:
!ls -hl data/halo_kin.00002.1e10_m*

-rw-rw-r-- 1 cadiou cadiou 140 Apr 13 17:21 data/halo_kin.00002.1e10_m_1e12.dat.tmp
-rw-rw-r-- 1 cadiou cadiou 98M May 18 17:37 data/halo_kin.00002.1e10_m_completed.csv
-rw-rw-r-- 1 cadiou cadiou 41M May 12 11:02 data/halo_kin.00002.1e10_m.dat
-rw-rw-r-- 1 cadiou cadiou 61M May 18 17:37 data/halo_kin.00002.1e10_m.hdf
